In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import jax
from jax import numpy as jnp
from jax.experimental import host_callback

from tqdm.auto import tqdm
from functools import partial
import optax
import matplotlib.pyplot as plt
from pprint import pprint

from flax import linen as nn
from typing import Sequence

from functools import partial
from typing import Callable, Sequence
from flax import struct
from flax import linen as nn
import jax
import jax.numpy as jnp
import optax
from functools import partial
import numpy as np

import actions
import qlearning
import replay
import plotting
from utils import ConvNet, Transition
from frozen_lake import EnvState, FrozenLake, ObsType, ActType, RNGKey

from mango_utils import MultiDQLTrainState
from mango_utils import eps_greedy_rollout, aggregate
import sim
import optuna


In [2]:
jax.devices()

[cuda(id=0)]

In [3]:
def objective(trial):
    seed = 42
    map_size = 4
    max_interactions = 64 * 1000
    map_scale = int(jnp.log2(map_size))

    # hyperparams
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    rollout_length = 2 * map_size**2
    n_rollouts = max_interactions // rollout_length
    train_iter = 8

    # setup annealing schedules
    annealing_durations = [
        int(train_iter * n_rollouts * trial.suggest_float(f"annealing_{i}", 0.1, 1.0 / map_scale))
        for i in range(map_scale)
    ]
    annealing_starts = jnp.cumsum(jnp.array([0] + annealing_durations[:-1]))
    annealing_schedules = [
        optax.linear_schedule(1.0, 0.0, dur, start)
        for dur, start in zip(annealing_durations, annealing_starts)
    ]

    @jax.jit
    def annealing_fn(step):
        return jnp.array([schedule(step) for schedule in annealing_schedules])

    run_simulation = partial(
        sim.run_mango_simulation,
        map_size=map_size,
        n_rollouts=n_rollouts,
        rollout_length=rollout_length,
        train_iter=train_iter,
        lr=lr,
        annealing_fn=annealing_fn,
    )
    rng_key = jax.random.PRNGKey(seed)
    env, dql_state, results = jax.vmap(run_simulation)(jax.random.split(rng_key, 8))

    accuracy = results.eval_reward[:, :, :, 0, -1].mean()
    return accuracy


study = optuna.create_study(
    study_name="mango",
    storage="sqlite:///mango.db",
    load_if_exists=True,
    direction="maximize"
)

study.optimize(objective, n_trials=100, show_progress_bar=True)

[I 2024-02-07 08:14:32,193] A new study created in RDB with name: mango


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

[I 2024-02-07 08:15:33,500] Trial 0 finished with value: 0.025859376415610313 and parameters: {'lr': 0.0008195194117554066, 'annealing_0': 0.38484120940426425, 'annealing_1': 0.35573267121612584}. Best is trial 0 with value: 0.025859376415610313.


  0%|          | 0/2000 [00:00<?, ?it/s]

...

[I 2024-02-07 09:36:10,453] Trial 97 finished with value: 0.07147461175918579 and parameters: {'lr': 0.00011288701730408318, 'annealing_0': 0.15322918134435504, 'annealing_1': 0.24285780256313227}. Best is trial 94 with value: 0.0891406312584877.


  0%|          | 0/2000 [00:00<?, ?it/s]

[I 2024-02-07 09:37:01,021] Trial 98 finished with value: 0.07850586622953415 and parameters: {'lr': 0.0001404325310947836, 'annealing_0': 0.10466484870974527, 'annealing_1': 0.19411285452271443}. Best is trial 94 with value: 0.0891406312584877.


  0%|          | 0/2000 [00:00<?, ?it/s]

[I 2024-02-07 09:37:52,147] Trial 99 finished with value: 0.06842773407697678 and parameters: {'lr': 0.00010363507109447253, 'annealing_0': 0.16781346561389346, 'annealing_1': 0.282342209668513}. Best is trial 94 with value: 0.0891406312584877.


In [ ]:
!optuna-dashboard sqlite:///mango.db